<a href="https://colab.research.google.com/github/suyash091/EEG-MULTIPLE-CHANNEL/blob/master/1%20channel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



```
# Mindwave | 1 channel | 512 sampling rate
```



In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
from os.path import isfile
import zipfile
import numpy as np
import random

dataset_path = './data/MW.zip'

def get_dataset_file():
    if not isfile(dataset_path):
        import urllib
        origin = (
            'http://www.mindbigdata.com/opendb/MindBigData-MW-v1.0.zip'
        )
        print('Downloading data from %s' % origin)
        urlretrieve(origin, dataset_path)
    return open(dataset_path, 'rb')

def get_datasets():
    f = get_dataset_file()
    zf = zipfile.ZipFile(f)
        
    data = [ line for line in zf.open('MW.txt')]
    entire_dataset = []
    current_event = np.zeros(1024 * 1 + 2)
    
    print('Reading data file')
    i = 0

    for l in data:
        #print(str(l))
        ids, event, device, channel, code, size, data = l.decode("utf-8").split('\t')

        signals = np.array([float(val) for val in data.split(',')])
        
        current_event[1+ i*1024:1+ i*1024 + min(len(signals), 1024)] = signals[:1024]
        i += 1

        if i == 1: # we assume all channels from an event are in sequence
            current_event[-1] = int(code)
            current_event[0] = min(len(signals), 1024)

            entire_dataset.append(current_event)
            current_event = np.zeros(1024 * 1 + 2)
            i = 0

    random.seed(111) # deterministic
    random.shuffle(entire_dataset)

    entire_dataset = np.array(entire_dataset)
    return entire_dataset[:60000], entire_dataset[60000:]

def split_into_subsequences(data, n_sequences, length):
    output = np.zeros((data.shape[0]*n_sequences, length*14+1))
    for i in range(data.shape[0]):
        if n_sequences == 1:
            steps = 0
        else:
            steps = (data[i, 0] - length) / (n_sequences - 1)

        for j in range(n_sequences):
            output[i*n_sequences+j,:length]           = data[i, j*steps: j*steps+length]
            output[i*n_sequences+j,length:length*2]   = data[i, j*steps+512: j*steps+512+length]
            output[i*n_sequences+j,length*2:length*3] = data[i, j*steps+512*2: j*steps+512*2+length]
            output[i*n_sequences+j,length*3:length*4] = data[i, j*steps+512*3: j*steps+512*3+length]
            output[i*n_sequences+j,-1] = data[i, -1]

    return output

In [0]:

import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
from sklearn.neighbors import KNeighborsClassifier
from sklearn.decomposition import PCA
from sklearn import metrics
from sklearn.metrics import confusion_matrix
from urllib.request import urlretrieve
#from sklearn.gaussian_process import GaussianProcess

In [0]:
(25/2050)*3586

43.73170731707317

In [0]:
train, test = get_datasets()
print(train.shape)
print(test.shape)

Reading data file
(60000, 1026)
(7635, 1026)


In [0]:
train[0].shape

(1026,)

In [0]:
len(train[-1])

1026

In [0]:
#train = train[:int(len(train)/3),:]
#test = test[:int(len(test)/3), :]

#train[train[:,-1] >= 0,-1] =  0
#test[test[:,-1] >= 0,-1] =  0

#train[:,-1] = train[:,-1] + 1
#test[:,-1] = test[:,-1] + 1

train_target = train[:,-1]
test_target = test[:,-1]

DECISIONBOUNDARY = 0

In [0]:
#FastFourierTransformation apply to the sets using the 4 sensors

train_f = np.zeros(train.shape)

for i in range(len(train_f)):
    length = train[i][0]
    train_f[i][0] = train[i][0]
    train_f[i][-1] = train[i][-1]
    
    for j in range(1):
        train_f[int(i)][int(1+1024*j):int(1+1024*j+length)] = np.abs(np.fft.fft(train[int(i)][int(1+1024*j):int(1+1024*j+length)]))
        
test_f = np.zeros(test.shape)

for i in range(len(test_f)):
    length = test[i,0]
    test_f[i,0] = test[i,0]
    test_f[i,-1] = test[i,-1]
    
    for j in range(1):
        test_f[i][int(1+1024*j):int(1+1024*j+length)] = np.abs(np.fft.fft(test[i][int(1+1024*j):int(1+1024*j+length)]))

In [0]:
#PCA with 25 components
pca = PCA(n_components = 100)
train_principal = pca.fit_transform(train_f)
test_principal = pca.transform(test_f)

In [0]:
test_principal.shape

(7635, 100)

In [0]:
#KNN
erreur = []
x = np.arange(1,20,1)
for n in (x):
    # train set
    neigh = KNeighborsClassifier(n_neighbors=n)
    neigh.fit(train_principal, train[:,-1])

    #test set
    total_correct = 0
    for i in range(test_principal.shape[0]):
        if neigh.predict(test_principal[i].reshape(1, -1))[0] == test[i,-1]:
            total_correct += 1

    erreur.append(float(total_correct) / test_principal.shape[0] * 100)
    print('Percentage correct %d:' % n, float(total_correct) / test_principal.shape[0] * 100)

Pourcentage correct 1: 27.557301899148655
Pourcentage correct 2: 27.2822527832351
Pourcentage correct 3: 27.622789783889978
Pourcentage correct 4: 27.570399476096924
Pourcentage correct 5: 27.753765553372627
Pourcentage correct 6: 27.295350360183367
Pourcentage correct 7: 27.518009168303863


KeyboardInterrupt: ignored

In [0]:
#Import models from scikit learn module:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import KFold   #For K-fold cross validation
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier, export_graphviz
from sklearn import metrics
import numpy as np
from sklearn.metrics import roc_curve, auc, precision_score, confusion_matrix, explained_variance_score, max_error, mean_absolute_error, mean_squared_error
from sklearn.linear_model import LogisticRegression, LinearRegression, Ridge, Lasso

In [0]:

#model = RandomForestClassifier(n_estimators=100)
#model.fit(train_principal,train[:,-1])


predictions = model.predict(test_principal)
prc=precision_score(predictions,test[:,-1], average=None)
cfm=confusion_matrix(predictions,test[:,-1])
accuracy = metrics.accuracy_score(predictions,test[:,-1])
print(prc,cfm,accuracy)
#print(explained_variance_score(predictions,test))
#print(max_error(predictions,test))
#print(mean_absolute_error(predictions, test, multioutput='raw_values'))
#print(mean_squared_error(predictions, test, multioutput='raw_values'))

[0.99799062 0.12055641 0.09937888 0.10651828 0.08085809 0.08417508
 0.11383538 0.09105691 0.09166667 0.07565789 0.06210191] [[1490    8   12   14    7   11    6   11   13    4   12]
 [   1   78   81   89   73   65   70   68   68   84   81]
 [   1   59   64   56   80   70   63   59   61   73   54]
 [   0   72   86   67   74   67   69   78   68   77   78]
 [   0   63   78   56   49   55   70   56   62   56   62]
 [   1   70   56   52   52   50   49   67   37   51   48]
 [   0   71   50   72   72   58   65   72   78   66   71]
 [   0   65   58   57   46   58   51   56   59   54   66]
 [   0   57   56   54   68   66   43   58   55   46   56]
 [   0   53   67   52   41   53   46   44   50   46   61]
 [   0   51   36   60   44   41   39   46   49   51   39]] 0.2696791093647675


In [0]:
model = LogisticRegression()
model.fit(train_principal,train[:,-1])


predictions = model.predict(test_principal)
prc=precision_score(predictions,test[:,-1], average=None)
cfm=confusion_matrix(predictions,test[:,-1])
accuracy = metrics.accuracy_score(predictions,test[:,-1])
print(prc,cfm,accuracy)

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


[1.         0.         0.01397516 0.09062003 0.00165017 0.02861953
 0.34150613 0.12520325 0.14666667 0.15460526 0.12738854] [[1493    0    1    0    0    0    0    0    0    0    0]
 [   0    0    0    0    0    0    0    0    0    0    0]
 [   0    8    9   10    9   14    4   10   13   11    8]
 [   0   56   66   57   58   60   52   69   64   70   54]
 [   0    3    1    3    1    1    3    1    1    2    3]
 [   0   10    9    9    8   17    9    4    6    8    6]
 [   0  207  196  203  192  190  195  198  183  182  208]
 [   0   95   77   79   75   63   63   77   64   77   82]
 [   0   80   98   84   91   78   73   73   88   91   75]
 [   0  116  116  106   97   86  110  110  114   94  112]
 [   0   72   71   78   75   85   62   73   67   73   80]] 0.2764898493778651


/usr/local/lib/python3.6/dist-packages/sklearn/svm/base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


In [0]:
model = LinearRegression()
#model.fit(train_principal,train[:,-1])


#predictions = model.predict(test_principal)
#prc=precision_score(predictions,test[:,-1], average=None)
print(explained_variance_score(predictions,test[:,-1]))
print(max_error(predictions,test[:,-1]))
print(mean_absolute_error(predictions, test[:,-1], multioutput='raw_values'))
print(mean_squared_error(predictions, test[:,-1], multioutput='raw_values'))

0.046288236316122555
9.0
[2.55769483]
[12.22946955]


In [0]:
model = RandomForestRegressor()
model.fit(train_principal,train[:,-1])


predictions = model.predict(test_principal)
print(explained_variance_score(predictions,test[:,-1]))
print(max_error(predictions,test[:,-1]))
print(mean_absolute_error(predictions, test[:,-1], multioutput='raw_values'))
print(mean_squared_error(predictions, test[:,-1], multioutput='raw_values'))

/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


-0.3970549932523322
7.4
[2.11721022]
[7.54174853]


In [0]:
from __future__ import absolute_import, division, print_function, unicode_literals

import tensorflow as tf
import numpy as np

!pip install scipy
from scipy import signal

In [15]:
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Dense(1024, input_dim=100, activation='relu'))
model.add(tf.keras.layers.Dense(4096, activation='relu'))
model.add(tf.keras.layers.Dropout(0.25))
model.add(tf.keras.layers.Dense(4096, activation='relu'))
model.add(tf.keras.layers.Dropout(0.25))
model.add(tf.keras.layers.Dense(4096, activation='relu'))
model.add(tf.keras.layers.Dense(11, activation='softmax'))
alpha = 2e-3
batch_size = 256
num_epochs = 3000
model.compile(
      optimizer=tf.train.GradientDescentOptimizer(learning_rate=alpha, ),
      loss=tf.keras.losses.sparse_categorical_crossentropy,
      metrics=['sparse_categorical_accuracy', 'accuracy']
  )

History = model.fit(
      x = train_principal,
      y = train[:,-1],
      batch_size = batch_size,
      epochs=num_epochs,
      #validation_data = (x_valid, y_valid),
      #callbacks = [checkPointer,tensorBoard]
  )
predictions = model.predict(test_principal)

Train on 60000 samples
Epoch 1/3000
60000/60000 [==============================] - 8s 139us/sample - loss: nan - sparse_categorical_accuracy: 0.0813 - acc: 0.0813
Epoch 2/3000
60000/60000 [==============================] - 8s 136us/sample - loss: nan - sparse_categorical_accuracy: 0.0814 - acc: 0.0814
Epoch 3/3000
60000/60000 [==============================] - 8s 136us/sample - loss: nan - sparse_categorical_accuracy: 0.0814 - acc: 0.0814
Epoch 4/3000
60000/60000 [==============================] - 8s 137us/sample - loss: nan - sparse_categorical_accuracy: 0.0814 - acc: 0.0814
Epoch 5/3000
60000/60000 [==============================] - 8s 137us/sample - loss: nan - sparse_categorical_accuracy: 0.0814 - acc: 0.0814
Epoch 6/3000
16640/60000 [=======>......................] - ETA: 5s - loss: nan - sparse_categorical_accuracy: 0.0802 - acc: 0.0802

KeyboardInterrupt: ignored